In [13]:
import pandas as pd
df = pd.read_csv('/home/vamsi/Documents/loan_defaulter_challenge/test_indessa.csv')

In [68]:
len(df['member_id'])

532428

In [42]:
df.columns.values

array(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'batch_enrolled', 'int_rate', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'application_type', 'verification_status_joint', 'last_week_pay',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim'], dtype=object)

In [43]:
white_space = []
for each_value in list(df.columns.values):
    if ' ' in df[each_value].value_counts():
        white_space.append(each_value)
white_space

['batch_enrolled']

In [14]:
def cleaning(df):
    df = df.fillna("0")
    df['last_week_pay'] = df['last_week_pay'].str.extract('(\d+)', expand=False)
    df = df.fillna("0")
    df['last_week_pay'] = df['last_week_pay'].astype(int)
    df['term'] = df['term'].str.extract('(\d+)', expand=False).astype(int)
    df = df.fillna("0")
    df['term'] = df['term'].astype(int)
    df.insert(0, 'payment_completion', (df['last_week_pay']/(df['term']/12*52+1))*100)
    df['payment_completion'] = df['payment_completion'].astype(int)
    df['zip_addr_code'] = df['zip_code'] + df['addr_state']
    df['zip_addr_code'] = df['zip_addr_code'].str.replace(r'xx', '').astype('object')
    
    features_list_all = list(df.columns.values)
    features_list = [feature for feature in features_list_all if feature not in ['batch_enrolled','desc','title','emp_title','zip_code','addr_state']]
    df_train = df[features_list]
    return df_train
if __name__ == '__main__':
    df_train_noise = cleaning(df)

In [71]:
len(df_train_noise.columns.values)

44

In [3]:
null_features = []
for each_column_name in list(df_train_noise.columns.values):
    if df_train_noise[each_column_name].isnull().values.any():
        null_features.append([each_column_name,df_train_noise[each_column_name].isnull().sum()])
null_features

[]

In [72]:
import numpy as np
from scipy.stats import mode
def filling_null_values(df,null_features):
    for each_feature in range(0,len(null_features)-3):
        if df[null_features[each_feature][0]].dtype == np.object: 
            df[null_features[each_feature][0]].fillna('none',inplace=True)
        else:
            df[null_features[each_feature][0]].fillna(0,inplace=True)
    for each_element in range(len(null_features)-3,len(null_features)):
        df[null_features[each_element][0]].fillna(mode(df[null_features[each_element][0]]).mode[0], inplace=True)
    return df
if __name__ == '__main__':
    df_free_from_null = filling_null_values(df_train_noise,null_features)

/home/vamsi/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [73]:
null_features = []
for each_column_name in list(df_train_noise.columns.values):
    if df_train_noise[each_column_name].isnull().values.any():
        null_features.append([each_column_name,df_train_noise[each_column_name].isnull().sum()])
null_features

[]

In [4]:
import numpy as np
categorical_variables_list = []
for each_variable in list(df_free_from_null.columns.values):
    if df_free_from_null[each_variable].dtype == np.object:
        categorical_variables_list.append(each_variable)
categorical_variables_list

NameError: name 'df_free_from_null' is not defined

In [5]:
import numpy as np
categorical_variables_list = []
for each_variable in list(df_train_noise.columns.values):
    if df_train_noise[each_variable].dtype == np.object:
        categorical_variables_list.append(each_variable)
categorical_variables_list

['grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'pymnt_plan',
 'purpose',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'application_type',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'zip_addr_code']

In [6]:
df_free_from_null = df_train_noise

In [10]:

df_train_noise.dtypes

payment_completion               int64
member_id                        int64
loan_amnt                        int64
funded_amnt                      int64
funded_amnt_inv                float64
term                             int64
int_rate                       float64
grade                            int64
sub_grade                        int64
emp_length                       int64
home_ownership                   int64
annual_inc                      object
verification_status             object
pymnt_plan                      object
purpose                         object
dti                            float64
delinq_2yrs                     object
inq_last_6mths                  object
mths_since_last_delinq          object
mths_since_last_record          object
open_acc                        object
pub_rec                         object
revol_bal                      float64
revol_util                      object
total_acc                       object
initial_list_status      

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for each_feature in categorical_variables_list:
    df_free_from_null[each_feature] = le.fit_transform(df_free_from_null[each_feature])
df_free_from_null.dtypes

TypeError: unorderable types: float() > str()

In [15]:
df_train_noise.to_csv('test_data_all_features.csv',sep=',',index=False)
